In [1]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/Quora-Questions/"

Mounted at /content/drive
 new_boolq.jsonl   quora_train.csv  'test (1).csv'   train.csv.zip
 quora_test.csv    results	     test.csv	     wiki-news-300d-1M.vec.zip


In [2]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
!pip install fuzzywuzzy
!pip install sentence_transformers
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import nltk
import time
from matplotlib.pyplot import figure
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
# import optuna
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import gc
from sklearn.model_selection import cross_val_score
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 13.5 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 6.5 MB 37.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=80aa15994773d967ada685e3a98a0ba217e6b67f2e199d9d550a9ca33a7e3c08
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
full_data = pd.read_csv('/content/drive/My Drive/Quora-Questions/train.csv.zip')
data, test = train_test_split(full_data, test_size=0.3)

In [4]:
print(data.columns)
print(data.is_duplicate.unique())
print(data.is_duplicate.value_counts())
print(data.shape)


Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')
[0 1]
0    178540
1    104463
Name: is_duplicate, dtype: int64
(283003, 6)


In [5]:
# Some general preprocessing notions 
def preprocess_text(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    x = re.sub(r"http\S+", "", x)
    x = re.sub('\W', ' ', x)
    
    lemmatizer = WordNetLemmatizer()
    x = lemmatizer.lemmatize(x)
    bfs = BeautifulSoup(x)
    x = bfs.get_text()
    x = x.strip()
    return x 
    

def data_cleaning(data):
    newdata = pd.DataFrame()
    newdata['question1_final'] = data.question1.apply(preprocess_text)
    newdata['question2_final'] = data.question2.apply(preprocess_text)
    newdata['is_duplicate'] = data.is_duplicate
    return newdata

traindata = data_cleaning(data)
testdata = data_cleaning(test)
traindata

,question1_final,question2_final,is_duplicate
404152,in the bible was the forbidden fruit a metaph...,what is the story of adam eve and the forbid...,0
134385,do vagina orgasms come naturally or is there a...,do vaginal orgasms come naturally or is there ...,1
314981,where can i found wonderful baking and most en...,where can i found adorable baked cupcakes in g...,1
331122,what does it feel like to be a top writer on q...,how does it feel to be a 2013 top writer,0
87088,what was it like to attend stanford with jonat...,i attend a public school while some of my best...,0
...,...,...,...
16943,what are some of the best google search tricks,what are some of the best tricks and tips for ...,1
75059,if the only group admin of a whatsapp group no...,where can i find work from home job in writing...,0
283616,being a non resident in usa how much tax do i ...,if i get 60k dollar in usa how much should i ...,1
182127,what are the distinctive features of the new y...,which new york city bars feature attractions f...,0


In [6]:
traindata = traindata.reset_index().drop(columns=['index'])
testdata = testdata.reset_index().drop(columns=['index'])

In [7]:
def doesMatch (q, match):
    q1, q2 = q['question1_final'], q['question2_final']
    q1 = q1.split()
    q2 = q2.split()
    if len(q1)>0 and len(q2)>0 and q1[match]==q2[match]:
        return 1
    else:
        return 0

# Features from predefined notebook (manually extracted features)
def feature_extract(data):
    data['q1_word_num'] = data.question1_final.apply(lambda x: len(x.split()))
    data['q2_word_num'] = data.question2_final.apply(lambda x: len(x.split()))
    
    data['total_word_num'] = data['q1_word_num'] + data['q2_word_num']
    data['differ_word_num'] = abs(data['q1_word_num'] - data['q2_word_num'])
    data['same_first_word'] = data.apply(lambda x: doesMatch(x, 0) ,axis=1)
    data['same_last_word'] = data.apply(lambda x: doesMatch(x, -1) ,axis=1)
    
    data['common_word_num'] = data.apply(lambda x: len(set(x.question1_final.split()).intersection(set(x.question2_final.split()))) ,axis=1)

    data["fuzz_ratio"] = data.apply(lambda x: fuzz.ratio(x.question1_final, x.question2_final), axis=1)
    data.fillna(0, inplace=True)
    return data

In [8]:
traindata = feature_extract(traindata)
testdata = feature_extract(testdata)

In [9]:
traindata.to_csv('/content/quora_train.csv', encoding = 'utf-8-sig', index=False) 

In [10]:
testdata.to_csv('/content/quora_test.csv', encoding = 'utf-8-sig', index=False) 

In [ ]:
!cp quora_train.csv /content/drive/MyDrive/Quora-Questions/